# Pre-processing mouse single-nuclei RNA-seq data with kallisto and bustools

In this tutorial we will process the 10x dataset [1k Brain Nuclei from an E18 Mouse](https://support.10xgenomics.com/single-cell-gene-expression/datasets/2.1.0/nuclei_900) using kallisto bus and a custom built DNA and intron index for mouse. We will generate two matrices: one for spliced transcripts and one for unspliced transcripts, and sum them to obtain total nuclear transcripts.

In [1]:
!date

Thu Jan 16 15:52:05 UTC 2020


## Pre-processing

### Download the data

__Note:__ We use the `-O` option for `wget` to rename the files to easily identify them.

In [2]:
%%time
!wget https://caltech.box.com/shared/static/j337aflq9ublmwaripkepob41mr23216.txt -O checksums.txt
!wget https://caltech.box.com/shared/static/2j8shgwmalzcjawuow51678a8yssvdef.gz -O nuclei_900_S1_L001_R1_001.fastq.gz
!wget https://caltech.box.com/shared/static/k2yydqlz2jtckw1shk5h536mxn47nm9n.gz -O nuclei_900_S1_L001_R2_001.fastq.gz
!wget https://caltech.box.com/shared/static/tlqdm0w3tvy8ogyktsz7ahggwurc6kkj.gz -O nuclei_900_S1_L002_R1_001.fastq.gz
!wget https://caltech.box.com/shared/static/gqrvkqllr9d7zq4e3yfrng9kgfbejowe.gz -O nuclei_900_S1_L002_R2_001.fastq.gz

--2020-01-16 15:52:07--  https://caltech.box.com/shared/static/j337aflq9ublmwaripkepob41mr23216.txt
Resolving caltech.box.com (caltech.box.com)... 107.152.26.197, 107.152.27.197
Connecting to caltech.box.com (caltech.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/j337aflq9ublmwaripkepob41mr23216.txt [following]
--2020-01-16 15:52:12--  https://caltech.box.com/public/static/j337aflq9ublmwaripkepob41mr23216.txt
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/j337aflq9ublmwaripkepob41mr23216.txt [following]
--2020-01-16 15:52:12--  https://caltech.app.box.com/public/static/j337aflq9ublmwaripkepob41mr23216.txt
Resolving caltech.app.box.com (caltech.app.box.com)... 107.152.27.199, 107.152.26.199
Connecting to caltech.app.box.com (caltech.app.box.com)|107.152.27.199|:443... connected.
HTTP requ

Then, we verify the integrity of the files we downloaded to make sure they were not corrupted during the download.

In [3]:
!md5sum -c checksums.txt --ignore-missing

nuclei_900_S1_L001_R1_001.fastq.gz: OK
nuclei_900_S1_L001_R2_001.fastq.gz: OK
nuclei_900_S1_L002_R1_001.fastq.gz: OK
nuclei_900_S1_L002_R2_001.fastq.gz: OK


### Install `kb`

Install `kb` for running the **kallisto | bustools** workflow.

In [4]:
!pip install git+https://github.com/pachterlab/kb_python@count-kite

  Cloning https://github.com/pachterlab/kb_python (to revision count-kite) to /tmp/pip-req-build-00ne_j5p
  Running command git clone -q https://github.com/pachterlab/kb_python /tmp/pip-req-build-00ne_j5p
  Running command git checkout -b count-kite --track origin/count-kite
  Switched to a new branch 'count-kite'
  Branch 'count-kite' set up to track remote branch 'count-kite' from 'origin'.
     |████████████████████████████████| 51kB 1.8MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for kb-python: filename=kb_python-0.24.4-cp36-none-any.whl size=80991434 sha256=46c80d686746b8cc5f5ea19481297c38fcb7c928c8f472d75d4261ce74266483
  Stored in directory: /tmp/pip-ephem-wheel-cache-6ek0s5dl/wheels/8e/56/56/c89223de74af26792675e82f4bb5223e7cf0d653a33038e34c
  Created wheel for loompy: filename=loompy-3.0.6-cp36-none-any.whl size=47896 sha256=5aa0e8271e5ecd4

### Download mouse reference files and build the index

We build a mouse cDNA and intron index from the mouse genome and annotations provided by Ensembl.

In [5]:
%%time
!wget ftp://ftp.ensembl.org/pub/release-98/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa.gz
!wget ftp://ftp.ensembl.org/pub/release-98/gtf/mus_musculus/Mus_musculus.GRCm38.98.gtf.gz

--2020-01-16 15:56:57--  ftp://ftp.ensembl.org/pub/release-98/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa.gz
           => ‘Mus_musculus.GRCm38.dna.primary_assembly.fa.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.8
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.8|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-98/fasta/mus_musculus/dna ... done.
==> SIZE Mus_musculus.GRCm38.dna.primary_assembly.fa.gz ... 805984352
==> PASV ... done.    ==> RETR Mus_musculus.GRCm38.dna.primary_assembly.fa.gz ... done.
Length: 805984352 (769M) (unauthoritative)

Mus_musculus.GRCm38 100%[===================>] 768.65M  23.0MB/s    in 36s     

2020-01-16 15:57:35 (21.1 MB/s) - ‘Mus_musculus.GRCm38.dna.primary_assembly.fa.gz’ saved [805984352]

--2020-01-16 15:57:36--  ftp://ftp.ensembl.org/pub/release-98/gtf/mus_musculus/Mus_musculus.GRCm38.98.gtf.gz
          

Notice we use the `-n` option to split the index into multiple files. This helps reduce the maximum memory usage so that Google Colab doesn't run out of memory.

In [6]:
%%time
!kb ref -i index.idx -g t2g.txt -f1 cdna.fa -f2 intron.fa \
-c1 cdna_t2c.txt -c2 intron_t2c.txt --workflow lamanno -n 8 \
Mus_musculus.GRCm38.dna.primary_assembly.fa.gz Mus_musculus.GRCm38.98.gtf.gz

[2020-01-16 15:57:41,860]    INFO Preparing Mus_musculus.GRCm38.dna.primary_assembly.fa.gz, Mus_musculus.GRCm38.98.gtf.gz
[2020-01-16 15:57:41,860]    INFO Decompressing Mus_musculus.GRCm38.dna.primary_assembly.fa.gz to tmp
[2020-01-16 15:58:08,556]    INFO Sorting tmp/Mus_musculus.GRCm38.dna.primary_assembly.fa to /content/tmp/tmptuipvatl
[2020-01-16 16:05:44,930]    INFO Decompressing Mus_musculus.GRCm38.98.gtf.gz to tmp
[2020-01-16 16:05:48,839]    INFO Sorting tmp/Mus_musculus.GRCm38.98.gtf to /content/tmp/tmpqc7d85yg
[2020-01-16 16:06:50,352]    INFO Splitting genome tmp/Mus_musculus.GRCm38.dna.primary_assembly.fa into cDNA at /content/tmp/tmpzpi_mdzg
[2020-01-16 16:06:50,352] WARNING The following chromosomes were found in the FASTA but doens't have any "transcript" features in the GTF: GL456383.1, GL456367.1, GL456390.1, GL456389.1, GL456379.1, GL456387.1, GL456370.1, GL456394.1, GL456359.1, GL456392.1, GL456378.1, GL456213.1, GL456396.1, JH584300.1, GL456360.1, GL456393.1, JH58

### Generate RNA count matrices

The following command will generate an RNA count matrix of cells (rows) by genes (columns) in H5AD format, which is a binary format used to store [Anndata](https://anndata.readthedocs.io/en/stable/) objects. Notice we are providing the index and transcript-to-gene mapping we downloaded in the previous step to the `-i` and `-g` arguments respectively, as well as the transcripts-to-capture lists to the `-c1` and `-c2` arguments. Also, these reads were generated with the 10x Genomics Chromium Single Cell v2 Chemistry, hence the `-x 10xv2` argument. To view other supported technologies, run `kb --list`. Note the `--workflow nucleus` to indicate this is a single nucleus experiment.

__Note:__ If you would like a Loom file instead, replace the `--h5ad` flag with `--loom`. If you want to use the raw matrix output by `kb` instead of their H5AD or Loom converted files, omit these flags.

In [22]:
%%time
!kb count -i index.idx.0,index.idx.1,index.idx.2,index.idx.3,index.idx.4,index.idx.5,index.idx.6,index.idx.7 \
-g t2g.txt -c1 cdna_t2c.txt -c2 intron_t2c.txt -x 10xv2 -o output -t 2 --workflow nucleus --h5ad \
nuclei_900_S1_L001_R1_001.fastq.gz nuclei_900_S1_L001_R2_001.fastq.gz \
nuclei_900_S1_L002_R1_001.fastq.gz nuclei_900_S1_L002_R2_001.fastq.gz

[2020-01-16 21:26:55,274]    INFO Generating BUS file using 8 indices
[2020-01-16 21:26:55,274]    INFO Generating BUS file to output/tmp/bus_part0 from
[2020-01-16 21:26:55,274]    INFO         nuclei_900_S1_L001_R1_001.fastq.gz
[2020-01-16 21:26:55,274]    INFO         nuclei_900_S1_L001_R2_001.fastq.gz
[2020-01-16 21:26:55,274]    INFO         nuclei_900_S1_L002_R1_001.fastq.gz
[2020-01-16 21:26:55,274]    INFO         nuclei_900_S1_L002_R2_001.fastq.gz
[2020-01-16 21:26:55,275]    INFO Using index index.idx.0
[2020-01-16 21:38:46,592]    INFO Generating BUS file to output/tmp/bus_part1 from
[2020-01-16 21:38:46,592]    INFO         nuclei_900_S1_L001_R1_001.fastq.gz
[2020-01-16 21:38:46,592]    INFO         nuclei_900_S1_L001_R2_001.fastq.gz
[2020-01-16 21:38:46,593]    INFO         nuclei_900_S1_L002_R1_001.fastq.gz
[2020-01-16 21:38:46,593]    INFO         nuclei_900_S1_L002_R2_001.fastq.gz
[2020-01-16 21:38:46,593]    INFO Using index index.idx.1
[2020-01-16 21:54:34,715]    INF

## Load the anndata into python

`kb` automatically sums the counts of genes for barcodes common to both spliced and unspliced matrices as an Anndata object in H5AD format (because `--h5ad` was used).

In [0]:
import anndata

In [0]:
adata = anndata.read('output/counts_unfiltered/adata.h5ad')

In [19]:
adata

AnnData object with n_obs × n_vars = 175538 × 55421 

In [20]:
adata.obs

""
index
AAACCTGAGAAAGTGG
AAACCTGAGAAGCCCA
AAACCTGAGAATTCCC
AAACCTGAGACACGAC
AAACCTGAGACCCACC
...
TTTGTCATCTTACCGC
TTTGTCATCTTAGCCC
TTTGTCATCTTCAACT


In [21]:
adata.var

""
index
ENSMUSG00000026535.9
ENSMUSG00000026315.13
ENSMUSG00000000817.10
ENSMUSG00000063558.4
ENSMUSG00000001138.13
...
ENSMUSG00000118447.1
ENSMUSG00000118422.1
ENSMUSG00000118472.1
